<a href="https://colab.research.google.com/github/dickymiswardi/workshop/blob/main/zqt020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# @title 1. INSTAL TOOLS FOR INPUT TEKS QURAN WITH VN VIDEO EDITOR BY DICKY MISWARDI { display-mode: "form" }
# @markdown Klik untuk memproses

from tqdm.notebook import tqdm
import subprocess
import time

# Daftar perintah instalasi
commands = [
    ("Menginstal tqdm", ['pip', 'install', 'tqdm']),
    ("Menginstal whisper", ['pip', 'install', 'git+https://github.com/openai/whisper.git']),
    ("Memperbarui apt", ['sudo', 'apt', 'update']),
    ("Menginstal ffmpeg", ['sudo', 'apt', 'install', 'ffmpeg', '-y']),
    ("Menginstal requests", ['pip', 'install', 'requests']),
    ("Menginstal ipywidgets", ['pip', 'install', 'ipywidgets'])
]

# Total jumlah perintah
total_commands = len(commands)

# Progress bar
with tqdm(total=100, desc="Proses Instalasi", bar_format="{l_bar}{bar} [ {elapsed} < {remaining} ]") as pbar:
    for idx, (description, command) in enumerate(commands):
        print(description)
        subprocess.run(command)
        # Update progress bar sesuai dengan persentase selesai
        pbar.update(100 / total_commands)
        time.sleep(1)  # Simulasi waktu proses jika diperlukan

# Pesan setelah proses instalasi selesai
print("\nLinks:")
print("1. Facebook: https://facebook.com/dickymiswardi")
print("2. Download font teks Qur'an: https://s.id/hafsfont")
print("3. Download VN Video Editor: https://s.id/vlognow")

Proses Instalasi:   0%|           [ 00:00 < ? ]

Menginstal tqdm
Menginstal whisper
Memperbarui apt
Menginstal ffmpeg
Menginstal requests
Menginstal ipywidgets

Links:
1. Facebook: https://facebook.com/dickymiswardi
2. Download font teks Qur'an: https://s.id/hafsfont
3. Download VN Video Editor: https://s.id/vlognow


In [9]:
# @title 2. TRANSKRIP AUDIO & VIDEO MENJADI TEKS { display-mode: "form" }
# @markdown Klik untuk memproses

import whisper
from tqdm import tqdm
import requests
import time
import threading
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import files

# Function to download the audio file with progress bar
def download_audio(url, file_name):
    response = requests.get(url, stream=True)
    total_length = int(response.headers.get('content-length', 0))
    block_size = 4096
    progress_bar = tqdm(total=total_length, unit='iB', unit_scale=True, desc="Downloading")

    with open(file_name, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    if total_length != 0 and progress_bar.n != total_length:
        print("ERROR, something went wrong")

# Function to convert seconds to hh:mm:ss,ms format
def format_time(seconds):
    milliseconds = int((seconds - int(seconds)) * 1000)
    seconds = int(seconds)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

# Function to transcribe audio and save as SRT with progress bar
def transcribe_audio_to_srt(file_name, srt_file_name, gif_widget):
    model = whisper.load_model("medium")

    # Load and process the audio with progress bar
    audio = whisper.load_audio(file_name)
    duration = audio.shape[0] / whisper.audio.SAMPLE_RATE

    print("Transcribing audio file...")
    pbar = tqdm(total=100, desc="Transcribing", unit="%")

    result = [None]  # Use list to store result in the thread

    def transcribe():
        result[0] = model.transcribe(file_name, language='ar')
        pbar.n = 100
        pbar.refresh()
        pbar.close()
        gif_widget.value = ""

    def update_progress():
        while pbar.n < 100:
            time.sleep(1)
            pbar.update(1)
            pbar.refresh()

    transcribe_thread = threading.Thread(target=transcribe)
    progress_thread = threading.Thread(target=update_progress)

    transcribe_thread.start()
    progress_thread.start()

    transcribe_thread.join()
    progress_thread.join()

    segments = result[0]['segments']
    with open(srt_file_name, 'w') as srt_file:
        for i, segment in enumerate(segments):
            start_time = format_time(segment['start'])
            end_time = format_time(segment['end'])
            text = segment['text']

            srt_file.write(f"{i + 1}\n")
            srt_file.write(f"{start_time} --> {end_time}\n")
            srt_file.write(f"{text}\n\n")

    return srt_file_name

# @title Input audio URL and transcribe

# Text box for the audio URL
audio_url = widgets.Text(
    value='',
    placeholder='Enter the URL of audio/video file',
    description='MP3/4 URL:',
    disabled=False
)

# Play button to start the process
button_url = widgets.Button(description="Transcribe URL")
button_upload = widgets.Button(description="Upload File & Transcribe")

# GIF widget
gif_widget = widgets.HTML(value="")

def on_button_url_clicked(b):
    gif_widget.value = '<img src="https://raw.githubusercontent.com/dickymiswardi/colorbox/main/animasi.gif" alt="Processing..." style="width:100px;height:100px;">'
    srt_file_name = "output_subtitles.srt"

    if audio_url.value:
        file_name = "audio_to_transcribe.mp3"
        # Download the audio file
        print("Downloading audio file...")
        download_audio(audio_url.value, file_name)
    else:
        print("No audio source provided.")
        return

    # Transcribe the audio file and save as SRT
    srt_file = transcribe_audio_to_srt(file_name, srt_file_name, gif_widget)
    print(f"Transcription complete! SRT file saved as {srt_file}")

    # Notify that all processes are complete
    print("Proses selesai!")

def on_button_upload_clicked(b):
    gif_widget.value = '<img src="https://raw.githubusercontent.com/dickymiswardi/colorbox/main/animasi.gif" alt="Processing..." style="width:100px;height:100px;">'
    srt_file_name = "output_subtitles.srt"

    uploaded = files.upload()
    if uploaded:
        for fn in uploaded.keys():
            file_name = fn
            print(f'User uploaded file "{file_name}" with length {len(uploaded[fn])} bytes')

        # Transcribe the audio file and save as SRT
        srt_file = transcribe_audio_to_srt(file_name, srt_file_name, gif_widget)
        print(f"Transcription complete! SRT file saved as {srt_file}")

        # Notify that all processes are complete
        print("Proses selesai!")
    else:
        print("No file uploaded.")

button_url.on_click(on_button_url_clicked)
button_upload.on_click(on_button_upload_clicked)

display(audio_url)
display(button_url)
display(button_upload)
display(widgets.HTML(""))  # Add a line break after the buttons
display(gif_widget)

Text(value='', description='MP3/4 URL:', placeholder='Enter the URL of audio/video file')

Button(description='Transcribe URL', style=ButtonStyle())

Button(description='Upload File & Transcribe', style=ButtonStyle())

HTML(value='')

HTML(value='')

Downloading: 100%|██████████| 14.6M/14.6M [00:00<00:00, 117MiB/s]


Transcribing audio file...


Transcribing: 100%|██████████| 100/100 [00:45<00:00,  2.20%/s]


Transcription complete! SRT file saved as output_subtitles.srt
Proses selesai!


In [10]:
# @title 3. MENGKONVERSI HASIL TRANSKRIP AUDIO & VIDEO MENJADI TEKS QURAN RASM UTSMANI { display-mode: "form" }
# @markdown Audio & video yang ditranskrip tadi termasuk surat dan ayat berapa di dalam Al-Quran? Klik untuk menentukan

import re
import pandas as pd
import requests
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Variabel global untuk menyimpan pilihan
selected_surah = None
selected_from_ayat = None
selected_to_ayat = None

# Fungsi untuk memparsing file SRT
def parse_srt(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()
    # Split by double newlines to separate each subtitle block
    blocks = content.split('\n\n')
    subtitles = []
    for block in blocks:
        lines = block.split('\n')
        if len(lines) >= 3:
            # The subtitle timing is in the second line, and the text is in the following lines
            text = ' '.join(lines[2:])
            # Remove any HTML tags or special characters
            text = re.sub(r'<.*?>', '', text)
            # Remove non-alphanumeric characters except spaces
            text = re.sub(r'[^\w\s]', '', text)
            subtitles.append(text)
    return subtitles

# Fungsi untuk menghitung jumlah kata
def count_words(text):
    # Split the text into words
    words = text.split()
    return len(words)

# Fungsi untuk menghitung jumlah kata dari setiap subtitle
def calculate_word_counts(subtitles):
    word_counts = []
    for text in subtitles:
        word_count = count_words(text)
        word_counts.append(word_count)
    return word_counts

#@title Contoh penggunaan
#@markdown

filename = '/content/output_subtitles.srt'
subtitles = parse_srt(filename)
word_counts = calculate_word_counts(subtitles)

# Mengonversi hasil word_counts ke format string yang diperlukan
word_counts_str = ', '.join(map(str, word_counts))

# Fungsi untuk mengambil ayat Quran Rasm Utsmani
def get_quran_ayats(surah_number, from_ayat, to_ayat, splits):
    combined_text = ""
    arabic_numerals = ["٠", "١", "٢", "٣", "٤", "٥", "٦", "٧", "٨", "٩"]

    def convert_to_arabic_number(num):
        reversed_num = str(num)[::-1]
        return ''.join(arabic_numerals[int(digit)] for digit in reversed_num)

    def split_text(text, splits):
        words = text.split()
        result = []
        index = 0
        for count in splits:
            segment = ' '.join(words[index:index + count])
            result.append(segment)
            index += count
        return '\n\n'.join(result)

    for ayat_number in range(from_ayat, to_ayat + 1):
        url = f"https://api.quran.com/api/v4/quran/verses/uthmani?verse_key={surah_number}:{ayat_number}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            ayat_text = data['verses'][0]['text_uthmani']
            ayat_text = ayat_text.replace("ْ", "ۡ").replace("أُو۟", "أُوْ").replace("ا۟", "اْ").replace("لَـَٔا", "لَأٓ").replace("لۡـَٔا", "لۡأٓ").replace("لِيَسُـۥٓـُٔ", "لِيَسُـُٔ").replace("۞", "")
            ayat_text += convert_to_arabic_number(ayat_number)
            combined_text += " " + ayat_text

    combined_text = combined_text.replace(" ۖ", "ۖ").replace(" ۚ", "ۚ").replace(" ۗ", "ۗ").replace(" ۛ", "ۛ").replace(" ۙ", "ۙ").replace(" ۘ", "ۘ").replace(" ۜ", "ۜ").replace(" ۩", "۩").strip()
    result_text = split_text(combined_text, splits)

    return result_text

# Fungsi untuk menggantikan teks dalam file SRT
def replace_srt_text(srt_file_path, new_text):
    with open(srt_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    new_lines = []
    text_index = 0
    text_parts = new_text.split('\n\n')

    for line in lines:
        if line.strip().isdigit():
            new_lines.append(line)
        elif '-->' in line:
            new_lines.append(line)
            if text_index < len(text_parts):
                new_lines.append(text_parts[text_index] + '\n\n')
                text_index += 1
        else:
            continue

    output_srt_file = 'output_ayat.srt'
    with open(output_srt_file, 'w', encoding='utf-8') as file:
        file.writelines(new_lines)

    return ''.join(new_lines)

# Fungsi untuk mengambil data surah dan ayat dari API
def fetch_surah_ayah_data():
    surah_response = requests.get("https://api.quran.com/api/v4/chapters")
    if surah_response.status_code == 200:
        surah_data = surah_response.json()
        surah_list = [(surah["id"], surah["name_simple"]) for surah in surah_data["chapters"]]
        return surah_list
    else:
        print("Error fetching surah data")
        return []

# Fungsi untuk menampilkan widget parameter surah dan ayat
def show_parameters():
    surah_list = fetch_surah_ayah_data()
    surah_options = [(f"{surah_name} ({surah_id})", surah_id) for surah_id, surah_name in surah_list]

    surah_number_widget = widgets.Dropdown(options=surah_options, description='Surah:')
    from_ayat_widget = widgets.Dropdown(description='From Ayat:', options=[1], disabled=True)
    to_ayat_widget = widgets.Dropdown(description='To Ayat:', options=[1], disabled=True)
    process_button = widgets.Button(description="Proses")
    gif_output = widgets.Output()

    def update_ayat_widgets(change):
        surah_id = change.new
        ayah_response = requests.get(f"https://api.quran.com/api/v4/chapters/{surah_id}")
        if ayah_response.status_code == 200:
            ayah_data = ayah_response.json()
            total_ayahs = ayah_data["chapter"]["verses_count"]
            ayat_options = list(range(1, total_ayahs + 1))
            from_ayat_widget.options = ayat_options
            to_ayat_widget.options = ayat_options
            from_ayat_widget.disabled = False
            to_ayat_widget.disabled = False
            to_ayat_widget.value = total_ayahs
        else:
            print("Error fetching ayah data")

    surah_number_widget.observe(update_ayat_widgets, names='value')

    display(surah_number_widget, from_ayat_widget, to_ayat_widget, process_button, widgets.HTML(""), gif_output)

    def on_button_click(b):
        global selected_surah, selected_from_ayat, selected_to_ayat
        with gif_output:
            clear_output(wait=True)
            display(HTML("<img src='https://raw.githubusercontent.com/dickymiswardi/colorbox/main/animasi.gif' width='100' height='100' />"))
        selected_surah = surah_number_widget.value
        selected_from_ayat = from_ayat_widget.value
        selected_to_ayat = to_ayat_widget.value
        splits = [int(x.strip()) for x in word_counts_str.split(',')]

        ayat_texts = get_quran_ayats(selected_surah, selected_from_ayat, selected_to_ayat, splits)

        # Gantikan teks dalam file SRT
        srt_file_path = '/content/output_subtitles.srt'
        new_srt_content = replace_srt_text(srt_file_path, ayat_texts)

        # Hapus animasi GIF dan tampilkan hasil
        clear_output(wait=True)

        # Menampilkan hasil dalam textarea
        display(HTML(f'''
            <textarea id="srtContent" style="width:100%;height:200px;">{new_srt_content}</textarea>
            <button onclick="copyToClipboard()">Copy</button>
            <p id="copyMessage" style="display:none;color:green;">Copied to clipboard!</p>
            <script>
            function copyToClipboard() {{
                var copyText = document.getElementById("srtContent");
                navigator.clipboard.writeText(copyText.value).then(function() {{
                    var copyMessage = document.getElementById("copyMessage");
                    copyMessage.style.display = "block";
                    setTimeout(function() {{
                        copyMessage.style.display = "none";
                    }}, 2000);
                }});
            }}
            </script>
        '''))

    process_button.on_click(on_button_click)

output = widgets.Output()
display(output)
show_parameters()


In [11]:
# @title 4. KOREKSI SUSUNAN KATA QURAN BERTAHAP { display-mode: "form" }
# @markdown Klik untuk memproses

import re
from IPython.display import display, HTML, clear_output
import requests
import ipywidgets as widgets

# Function to fetch file content from URL
def fetch_file_content(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text

# Load colorbox.js and colorbox.css content from GitHub URLs
colorbox_js = fetch_file_content('https://raw.githubusercontent.com/dickymiswardi/colorbox/main/jquery.colorbox.js')
colorbox_css = fetch_file_content('https://raw.githubusercontent.com/dickymiswardi/colorbox/main/example4/colorbox.css')

def read_srt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.readlines()

def find_and_update_first_subtitle(file_path, additional_file_path):
    lines = read_srt(file_path)
    additional_lines = read_srt(additional_file_path)

    updated_lines = []
    word_counts = []
    line_texts = []
    additional_texts = []
    additional_word_counts = []

    for line in lines:
        if line.strip().isdigit() or '-->' in line:
            updated_lines.append(line.strip())
            continue
        words = line.split()
        if words:
            word_counts.append(len(words))
            line_texts.append(line.strip())
            updated_lines.append(line.strip())

    for line in additional_lines:
        if line.strip().isdigit() or '-->' in line:
            continue
        additional_words = line.split()
        if additional_words:
            additional_word_counts.append(len(additional_words))
            additional_texts.append(line.strip())

    while len(additional_texts) < len(line_texts):
        additional_texts.append("")
        additional_word_counts.append("")

    table_rows = ""
    for idx, (count, text, add_count, additional_text) in enumerate(zip(word_counts, line_texts, additional_word_counts, additional_texts), start=1):
        table_rows += f"<tr><td>{idx}</td><td contenteditable='true' class='word-count' inputmode='numeric' pattern='[0-9]*'>{count}</td><td class='main-text'>{text}</td><td>{add_count}</td><td class='unclickable'>{additional_text}</td></tr>"

    display(HTML(f"""
        <style>
            .table-container {{
                position: relative;
                height: 400px;
                overflow-y: auto;
            }}
            .update-button {{
                position: sticky;
                top: 0;
                left: 0;
                z-index: 1;
            }}
            .main-text {{
                pointer-events: none;
                background-color: #f0f0f0;
            }}
            .unclickable {{
                pointer-events: none;
            }}
            {colorbox_css}
        </style>
        <div class="table-container">
            <button id="showColorboxButtonTop" class="update-button" onclick="showColorbox()">Download SRT</button>
            <button id="updateButtonTop" class="update-button" onclick="runKodeKedua()">Update Input Angka</button>
            <div>
                <p><strong>Penting! Tekan tombol "Update Input Angka" untuk mengaktifkan tombol "Download SRT."</strong></p>
                <p><strong>Penting! Tekan tombol "Run" (▶) utama cell untuk mengupdate table.</strong></p>
                <p>Masukkan angka pada "Edit Word Count" di "Tabel Koreksi" dan perbaiki sesuai komposisi kata Al-Quran pada durasi SRT kemudian klik tombol "Update Input Angka"</p>
                <table border="1" style="width:100%; border-collapse: collapse;" id="editable_table">
                    <thead>
                        <tr><th>No</th><th>Edit Word Count</th><th>Main Text</th><th>Original Word Count</th><th>Original Subtitles</th></tr>
                    </thead>
                    <tbody>
                        {table_rows}
                    </tbody>
                </table>
                <button id="updateButtonBottom" class="update-button" onclick="runKodeKedua()">Update Input Angka</button>
            </div>
        </div>
        <p id="runMessage" style="display:none;color:green;">Kode kedua berhasil dijalankan!</p>
        <button onclick="showColorbox()">Download SRT</button>
        <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
        <script>
        {colorbox_js}
        </script>
        <script>
        function saveTableToLocalStorage() {{
            var table = document.getElementById('editable_table');
            var rows = Array.from(table.rows).slice(1);
            var tableData = rows.map(row => {{
                return {{
                    no: row.cells[0].innerText,
                    wordCount: row.cells[1].innerText,
                    mainText: row.cells[2].innerText,
                    originalWordCount: row.cells[3].innerText,
                    originalText: row.cells[4].innerText
                }};
            }});
            localStorage.setItem('tableData', JSON.stringify(tableData));
        }}

        function loadTableFromLocalStorage() {{
            var tableData = JSON.parse(localStorage.getItem('tableData') || '[]');
            var table = document.getElementById('editable_table').getElementsByTagName('tbody')[0];
            tableData.forEach(data => {{
                var row = table.insertRow();
                row.insertCell(0).innerText = data.no;
                row.insertCell(1).innerText = data.wordCount;
                row.insertCell(2).innerText = data.mainText;
                row.insertCell(3).innerText = data.originalWordCount;
                row.insertCell(4).innerText = data.originalText;
            }});
        }}

        function validateWordCounts() {{
            var isValid = true;
            var wordCountCells = document.getElementsByClassName('word-count');
            for (var i = 0; i < wordCountCells.length; i++) {{
                var value = wordCountCells[i].innerText.trim();
                if (value === '' || value === '0' || !/^\d+$/.test(value)) {{
                    isValid = false;
                    break;
                }}
            }}
            return isValid;
        }}

        function runKodeKedua() {{
            if (!validateWordCounts()) {{
                $.colorbox({{html: '<p><strong>SRT BOX EDITOR</strong></p><p>Angka 0, koma, titik, huruf, simbol, atau membiarkan kolom kosong adalah tidak sah. Jika kolom ini mengandung basmalah atau taawudz tulis angka 1 kemudian edit di SRT BOX EDITOR dan klik Download Updated/Edited SRT.</p>', width: "90%", height: "90%", closeButton: true}});
                return;
            }}
            var updateButtonTop = document.getElementById('updateButtonTop');
            var updateButtonBottom = document.getElementById('updateButtonBottom');
            updateButtonTop.disabled = true;
            updateButtonTop.innerText = 'Processing...';
            updateButtonBottom.disabled = true;
            updateButtonBottom.innerText = 'Processing...';
            saveTableToLocalStorage();
            var tableData = JSON.parse(localStorage.getItem('tableData') || '[]');
            var splits = tableData.map(data => data.wordCount).join(', ');

            // Panggil fungsi Python untuk menjalankan kode kedua
            google.colab.kernel.invokeFunction('notebook.runKodeKedua', [splits], {{}});
        }}

        document.addEventListener('DOMContentLoaded', loadTableFromLocalStorage);
        document.getElementById('editable_table').addEventListener('input', saveTableToLocalStorage);

        function showColorbox() {{
            var width = $(window).width() > 500 ? "90%" : "100%";
            var height = $(window).height() > 500 ? "90%" : "100%";
            $.colorbox({{inline:true, href:"#inline_content", width:width, height:height, closeButton: true, close: "(X) Close dan Update Input Angka"}});
        }}
        </script>
    """))

# Paths to your SRT files
srt_file_path = '/content/output_ayat.srt'
additional_srt_file_path = '/content/output_subtitles.srt'
find_and_update_first_subtitle(srt_file_path, additional_srt_file_path)

def fetch_surah_ayah_data():
    surah_response = requests.get("https://api.quran.com/api/v4/chapters")
    if surah_response.status_code == 200:
        surah_data = surah_response.json()
        surah_list = [(surah["id"], surah["name_simple"]) for surah in surah_data["chapters"]]
        return surah_list
    else:
        print("Error fetching surah data")
        return []

surah_list = fetch_surah_ayah_data()
process_button = widgets.Button(description="Proses", layout=widgets.Layout(display='none'))
confirm_button = widgets.Button(description="Update Input Angka", layout=widgets.Layout(display='none'))
textarea_output = widgets.Output()

def run_kode_kedua(splits):
    global selected_surah, selected_from_ayat, selected_to_ayat
    splits = [int(x.strip()) for x in splits.split(',')]
    ayat_texts = get_quran_ayats(selected_surah, selected_from_ayat, selected_to_ayat, splits)

    srt_file_path = '/content/output_subtitles.srt'
    new_srt_content = replace_srt_text(srt_file_path, ayat_texts)

    with textarea_output:
        clear_output(wait=True)
        display(HTML(f'''
            <div style="display:none">
                <div id="inline_content" style="padding:10px; background:#fff;">
                    <p><strong>SRT BOX EDITOR</strong></p>
                    <p>أَعُوذُ بِٱللَّهِ مِنَ ٱلشَّیۡطَـٰنِ ٱلرَّجِیمِ <button onclick="copyText('auzubillah')">Copy</button> <span id="auzubillahMessage" style="display:none;color:green;">Copied to clipboard!</span><br>
                    بِسۡمِ ٱللَّهِ ٱلرَّحۡمَـٰنِ ٱلرَّحِیمِ <button onclick="copyText('bismillah')">Copy</button> <span id="bismillahMessage" style="display:none;color:green;">Copied to clipboard!</span></p>
                    <textarea id="srtContent" style="width:100%;height:200px;">{new_srt_content}</textarea>
                    <button onclick="copyToClipboard()">Copy</button> <span id="srtCopyMessage" style="display:none;color:green;">Copied to clipboard!</span>
                    <button onclick="downloadSRT()">Download Updated/Edited SRT</button>
                    <p>Anda bisa mengedit kembali susunan kata Al-Quran pada durasi SRT di box ini kemudian download.</p>
                    <script>
                    function copyToClipboard() {{
                        var copyText = document.getElementById("srtContent");
                        navigator.clipboard.writeText(copyText.value).then(function() {{
                            var copyMessage = document.getElementById("srtCopyMessage");
                            copyMessage.style.display = "inline";
                            setTimeout(function() {{
                                copyMessage.style.display = "none";
                            }}, 2000);
                        }});
                    }}

                    function copyText(type) {{
                        var text = "";
                        var message = "";
                        if (type === 'auzubillah') {{
                            text = 'أَعُوذُ بِٱللَّهِ مِنَ ٱلشَّیۡطَـٰنِ ٱلرَّجِیمِ';
                            message = document.getElementById("auzubillahMessage");
                        }} else if (type === 'bismillah') {{
                            text = 'بِسۡمِ ٱللَّهِ ٱلرَّحۡمَـٰنِ ٱلرَّحِیمِ';
                            message = document.getElementById("bismillahMessage");
                        }}
                        navigator.clipboard.writeText(text).then(function() {{
                            message.style.display = "inline";
                            setTimeout(function() {{
                                message.style.display = "none";
                            }}, 2000);
                        }});
                    }}

                    function downloadSRT() {{
                        var srtContent = document.getElementById("srtContent").value;
                        var blob = new Blob([srtContent], {{ type: 'text/plain' }});
                        var url = window.URL.createObjectURL(blob);
                        var a = document.createElement('a');
                        a.style.display = 'none';
                        a.href = url;
                        a.download = 'output_ayat.srt';
                        document.body.appendChild(a);
                        a.click();
                        window.URL.revokeObjectURL(url);
                    }}
                    </script>
                </div>
            </div>
            <script>
                $(document).ready(function(){{
                    var width = $(window).width() > 500 ? "90%" : "100%";
                    var height = $(window).height() > 500 ? "90%" : "100%";
                    $.colorbox({{inline:true, href:"#inline_content", width:width, height:height, closeButton: true, close: "(X) Close dan Update Input Angka"}});
                }});
            </script>
        '''))

    js_code = """
    var updateButtonTop = document.getElementById('updateButtonTop');
    var updateButtonBottom = document.getElementById('updateButtonBottom');
    updateButtonTop.disabled = false;
    updateButtonTop.innerText = 'Update Input Angka';
    updateButtonBottom.disabled = false;
    updateButtonBottom.innerText = 'Update Input Angka';
    """

    display(HTML(f'<script>{js_code}</script>'))

def on_confirm_button_clicked(splits):
    run_kode_kedua(splits)
    display(HTML("<script>document.getElementById('updateButtonTop').disabled = false; document.getElementById('updateButtonTop').innerText = 'Update Input Angka';</script>"))
    display(HTML("<script>document.getElementById('updateButtonBottom').disabled = false; document.getElementById('updateButtonBottom').innerText = 'Update Input Angka';</script>"))

from google.colab import output
output.register_callback('notebook.runKodeKedua', on_confirm_button_clicked)

confirm_button.on_click(lambda b: on_confirm_button_clicked(splits_input.value))

display(process_button, confirm_button, textarea_output)

def get_quran_ayats(surah_number, from_ayat, to_ayat, splits):
    combined_text = ""
    arabic_numerals = ["٠", "١", "٢", "٣", "٤", "٥", "٦", "٧", "٨", "٩"]

    def convert_to_arabic_number(num):
        reversed_num = str(num)[::-1]
        return ''.join(arabic_numerals[int(digit)] for digit in reversed_num)

    def split_text(text, splits):
        words = text.split()
        result = []
        index = 0
        for count in splits:
            segment = ' '.join(words[index:index + count])
            result.append(segment)
            index += count
        return '\n\n'.join(result)

    for ayat_number in range(from_ayat, to_ayat + 1):
        url = f"https://api.quran.com/api/v4/quran/verses/uthmani?verse_key={surah_number}:{ayat_number}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            ayat_text = data['verses'][0]['text_uthmani']
            ayat_text = ayat_text.replace("ْ", "ۡ")
            ayat_text = ayat_text.replace("أُو۟", "أُوْ")
            ayat_text = ayat_text.replace("ا۟", "اْ")
            ayat_text = ayat_text.replace("لَـَٔا", "لَأٓ")
            ayat_text = ayat_text.replace("لۡـَٔا", "لۡأٓ")
            ayat_text = ayat_text.replace("لِيَسُـۥٓـُٔ", "لِيَسُـُٔ")
            ayat_text = ayat_text.replace("۞", "")
            ayat_text += convert_to_arabic_number(ayat_number)
            combined_text += " " + ayat_text

    combined_text = combined_text.replace(" ۖ", "ۖ").replace(" ۚ", "ۚ").replace(" ۗ", "ۗ").replace(" ۛ", "ۛ").replace(" ۙ", "ۙ").replace(" ۘ", "ۘ").replace(" ۜ", "ۜ").replace(" ۩", "۩")

    combined_text = combined_text.strip()
    result_text = split_text(combined_text, splits)

    return result_text

def replace_srt_text(srt_file_path, new_text):
    with open(srt_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    new_lines = []
    text_index = 0
    text_parts = new_text.split('\n\n')

    for line in lines:
        if line.strip().isdigit():
            new_lines.append(line)
        elif '-->' in line:
            new_lines.append(line)
            if text_index < len(text_parts):
                new_lines.append(text_parts[text_index] + '\n\n')
                text_index += 1
        else:
            continue

    output_srt_file = 'output_ayat.srt'
    with open(output_srt_file, 'w', encoding='utf-8') as file:
        file.writelines(new_lines)

    return ''.join(new_lines)


No,Edit Word Count,Main Text,Original Word Count,Original Subtitles
1,4,هَلۡ أَتَىٰ عَلَى ٱلۡإِنسَـٰنِ,4,بسم الله الرحمن الرحيم
2,11,حِينٌ مِّنَ ٱلدَّهۡرِ لَمۡ يَكُن شَيۡـًٔا مَّذۡكُورًا١ إِنَّا خَلَقۡنَا ٱلۡإِنسَـٰنَ مِن,11,هل أتى على الإنسان حين من الدهر لم يكن شيئاً مذكوراً
3,7,نُّطۡفَةٍ أَمۡشَاجٍ نَّبۡتَلِيهِ فَجَعَلۡنَـٰهُ سَمِيعًۢا بَصِيرًا٢ إِنَّا,7,إنا خلقنا الإنسان من نطفة أمشاج نبتليه
4,3,هَدَيۡنَـٰهُ ٱلسَّبِيلَ إِمَّا,3,فجعلناه سميعاً بصيراً
5,7,شَاكِرًا وَإِمَّا كَفُورًا٣ إِنَّآ أَعۡتَدۡنَا لِلۡكَـٰفِرِينَ سَلَـٰسِلَاْ,7,إنا هديناه السبيل إما شاكراً وإما كفوراً
6,6,وَأَغۡلَـٰلًا وَسَعِيرًا٤ إِنَّ ٱلۡأَبۡرَارَ يَشۡرَبُونَ مِن,6,إنا أعتدنا للكافرين سلاسل وأغلالاً وسعيراً
7,8,كَأۡسٍ كَانَ مِزَاجُهَا كَافُورًا٥ عَيۡنًا يَشۡرَبُ بِهَا عِبَادُ,8,إن الأبرار يشربون من كأس كان مزاجها كافوراً
8,7,ٱللَّهِ يُفَجِّرُونَهَا تَفۡجِيرًا٦ يُوفُونَ بِٱلنَّذۡرِ وَيَخَافُونَ يَوۡمًا,7,عيناً يشرب بها عباد الله يفجرونها تفجيراً
9,7,كَانَ شَرُّهُۥ مُسۡتَطِيرًا٧ وَيُطۡعِمُونَ ٱلطَّعَامَ عَلَىٰ حُبِّهِۦ,7,يوفون بالنذر ويخافون يوماً كان شره مستطيراً
10,7,مِسۡكِينًا وَيَتِيمًا وَأَسِيرًا٨ إِنَّمَا نُطۡعِمُكُمۡ لِوَجۡهِ ٱللَّهِ,7,ويطعمون الطعام على حبيه مسكيناً ويتيماً وأسيراً


Button(description='Proses', layout=Layout(display='none'), style=ButtonStyle())

Button(description='Update Input Angka', layout=Layout(display='none'), style=ButtonStyle())

Output()

In [15]:
# @title 5. MENERJEMAHKAN AYAT QURAN KE BAHASA INDONESIA { display-mode: "form" }
# @markdown Menampilkan terjemahan ayat-ayat Al-Quran ke dalam Bahasa Indonesia berdasarkan input dari kode Python 1.

import requests
from IPython.display import display, clear_output, HTML

# Pastikan variabel global dari Kode Python 1 telah diatur
try:
    if selected_surah is None or selected_from_ayat is None or selected_to_ayat is None:
        raise NameError('Variabel global tidak diatur dengan benar dari Kode Python 1')
except NameError as e:
    print(e)

# Function to extract words from the response
def extract_words(ayah):
    words = ayah.get('words', [])
    word_texts = [word['text'] for word in words]
    return word_texts

# Function to fetch and process an ayah by its key
def fetch_ayah(ayah_key, params):
    endpoint = f"https://api.quran.com/api/v4/verses/by_key/{ayah_key}"
    response = requests.get(endpoint, params=params)
    if response.status_code == 200:
        ayah = response.json()['verse']
        return extract_words(ayah)
    else:
        print(f"Error fetching Ayah {ayah_key}: {response.status_code}")
        return None

# Parameters for the API request
params = {
    "language": "id",
    "words": "true",
    "fields": "words"
}

# Fungsi untuk memproses ayat berdasarkan input dari Kode Python 1
def process_ayahs():
    if selected_surah and selected_from_ayat and selected_to_ayat:
        with output:
            clear_output(wait=True)
            print(f"Menerjemahkan Surah {selected_surah} dari Ayat {selected_from_ayat} sampai Ayat {selected_to_ayat}...")

            for ayah_number in range(selected_from_ayat, selected_to_ayat + 1):
                ayah_key = f"{selected_surah}:{ayah_number}"
                words = fetch_ayah(ayah_key, params)
                if words:
                    # Concatenate last word with previous words
                    words[-2] += words[-1]
                    words.pop()
                    words_text = ' '.join(words)
                    print(words_text)
    else:
        with output:
            clear_output(wait=True)
            print("Data Surah dan Ayat belum lengkap.")

# Output widget untuk menampilkan hasil
output = widgets.Output()
display(output)

# Memproses ayat berdasarkan input dari Python 1
process_ayahs()


Output()